<a href="https://colab.research.google.com/github/QuratulAin20/AI-agents/blob/main/AI_AGENT_DESIGN_PATTERNS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reflection Agent
- It enable AI agents to reflect on their output by self-assessing their performance.

**Strategy**
- It uses prompting strategy to improve the quality and success rate of agents and similar AI systems.

**Working Method**
- It involves prompting an LLM to reflect on and critique its past actions, sometimes incorporating additional external information such as tool observations

*It has following 3 architecture*

## Basic Reflection Architecture
- It composes of two LLM calls: a generator and a reflector.
- The generator tries to respond directly to the user's requests.
-  The reflector is prompted to role play as a teacher and offer constructive criticism for the initial response.



## Reflexion
## Language Agents Tree Search







### Building basic reflection agent architecture

In [17]:
%pip install -U --quiet  langgraph
%pip install -U --quiet tavily-python
%pip install -qU langchain-groq


In [18]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)


_set_if_undefined("TAVILY_API_KEY")
_set_if_undefined("GROQ_API_KEY")

> Basic reflection architecture consists of generator and reflector so first we create our reflector

**Task is to generate an essasy on AI design patterns**

In [28]:
# Creating generator

# necessary imports
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq

# Defining the prompt for generator
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are an expert article writer for AI."
            "Write an article of 3 paragraphs"
            "write the best possible article on user request"
            "If user provide critique, response with the refined version of article",
         ),
        MessagesPlaceholder(variable_name = "messages"),

    ]
)

# define llm
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.5,
    max_tokens=500,
    timeout=None,
    max_retries=2,
)

# generation
generate = prompt|llm

In [29]:
# Getting response

article =""

request = HumanMessage(
    content = "write an article of AI agent architecture"
)

for chunk in generate.stream({"messages":[request]}):
  print(chunk.content, end="")
  article += chunk.content


**The Evolution of AI Agent Architecture: A Comprehensive Overview**

The development of Artificial Intelligence (AI) has led to the creation of sophisticated AI agents that can interact with humans and other systems. An AI agent is a software program that perceives its environment, processes information, and takes actions to achieve its goals. Over the years, AI agent architecture has undergone significant transformations, driven by advances in machine learning, natural language processing, and cognitive computing. In this article, we will delve into the evolution of AI agent architecture and explore the key components that make up a modern AI agent.

**Traditional AI Agent Architecture**

The traditional AI agent architecture is based on the Agent-Environment Interaction model, which consists of three main components: Perception, Action, and Knowledge. Perception involves sensing the environment and extracting relevant information, while Action involves taking decisions and executing

In [30]:
# Creating Reflector

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a crtique on an article writing"
            "Grade the article and generate critics and recommendation to improve the article on user submission"
            "Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name = "messages"),
    ]
)

# defining LLM
# we already define it

# creting reflector chain
reflect = reflection_prompt | llm

reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=article)]}):
    print(chunk.content, end="")
    reflection += chunk.content

**Grade:** B+ (87%)

**Critique:**

The article provides a good overview of the evolution of AI agent architecture, but it lacks depth and detail in certain areas. Here are some specific criticisms and recommendations for improvement:

1. **Length:** The article is a bit too short, and some sections feel rushed. Consider expanding the article to at least 1,000-1,500 words to provide a more comprehensive overview.
2. **Depth:** While the article provides a good high-level overview of AI agent architecture, it lacks depth in certain areas. For example, the article could benefit from more detailed explanations of machine learning and deep learning techniques, as well as more examples of how they are applied in AI agent architecture.
3. **Style:** The article is written in a clear and concise style, but it could benefit from more engaging and descriptive language. Consider using more vivid metaphors and analogies to help readers understand complex concepts.
4. **Organization:** The article

In [31]:
# Creating loop for repeat the task and get final output
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=article), HumanMessage(content=reflection)]}   # we are mimicking reflection agent as Human but in real it is AI agent
):
    print(chunk.content, end="")

**Refined Article: The Evolution of AI Agent Architecture: A Comprehensive Overview**

**Introduction**

Artificial Intelligence (AI) has revolutionized the way we interact with technology, and AI agents are at the heart of this revolution. An AI agent is a software program that perceives its environment, processes information, and takes actions to achieve its goals. From the early days of rule-based systems to the modern era of machine learning and deep learning, AI agent architecture has undergone significant transformations. In this article, we will delve into the evolution of AI agent architecture, exploring the key components that make up a modern AI agent and the benefits of this new paradigm.

**Traditional AI Agent Architecture**

The traditional AI agent architecture is based on the Agent-Environment Interaction model, which consists of three main components: Perception, Action, and Knowledge. Perception involves sensing the environment and extracting relevant information, whi

In [34]:
# Building graph for basic reflection archotecture, as we did in isolation above

from typing import Annotated, List, Sequence
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}


async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}

builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")

def should_continue(state: State):
    if len(state["messages"]) > 2:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

- `StateGraph` use if app require complex state management
- `MessageGraph` use if we want single conversational flow and simple routing decidion

- These are the classes langgraph provided to orchesterate the converational flow between different nodes.
- `messagegraph` maintain the list of messages and decided the flow of these messages netween nodes.
- Every node receive the list of previous messages.
- Each node can append the new message to the list and return it.


Sure! Let's break down the `reflection_node` function in simpler terms:

### Purpose of the Function
The `reflection_node` function is designed to take the existing messages, translate them into a format that the reflection process can understand, and then generate a response based on that.

### Step-by-Step Explanation

1. **Mapping Message Types**:
   ```python
   cls_map = {"ai": HumanMessage, "human": AIMessage}
   ```
   - Here, we create a dictionary called `cls_map` that tells us how to convert messages based on who sent them:
     - Messages from the AI are treated as `HumanMessage`.
     - Messages from the human are treated as `AIMessage`.

2. **Translating Messages**:
   ```python
   translated = [state["messages"][0]] + [
       cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
   ]
   ```
   - We start with the first message (the original user request) and keep it the same.
   - For the rest of the messages, we use the `cls_map` to convert them:
     - If a message is from the AI, it becomes a `HumanMessage`.
     - If it’s from the human, it becomes an `AIMessage`.
   - This creates a new list called `translated` that prepares the messages for the reflection process.

3. **Generating a Response**:
   ```python
   res = await reflect.ainvoke(translated)
   ```
   - We send the translated messages to the reflection process (`reflect.ainvoke`), which generates a response based on them.
   - The `await` keyword allows the program to wait for this process to finish without blocking other tasks.

4. **Returning the Response**:
   ```python
   return {"messages": [HumanMessage(content=res.content)]}
   ```
   - Finally, we wrap the response from the reflection process in a `HumanMessage` and return it in a dictionary format.

### Summary
The `reflection_node` function takes the existing messages, translates them into a suitable format for reflection, generates a new response based on that, and returns it. This helps create a more meaningful interaction between the user and the AI by reflecting on the conversation so far.

In [35]:

config = {"configurable": {"thread_id": "1"}}

In [36]:

async for event in graph.astream(
    {
        "messages": [
            HumanMessage(
                content="Generate an article on AI agent design pattern"
            )
        ],
    },
    config,
):
    print(event)
    print("---")


{'generate': {'messages': [AIMessage(content="**Designing AI Agents: A Comprehensive Guide to the AI Agent Pattern**\n\nArtificial intelligence (AI) has revolutionized the way we interact with machines, and AI agents are at the forefront of this revolution. An AI agent is a software program that perceives its environment, takes actions, and learns from its experiences to achieve a specific goal. In this article, we will delve into the design pattern of AI agents, exploring the key components, benefits, and applications of this powerful paradigm.\n\nAt its core, an AI agent consists of three primary components: perception, reasoning, and action. **Perception** involves the agent's ability to gather information from its environment, whether through sensors, APIs, or other data sources. **Reasoning** is the agent's decision-making process, which can be based on rules, machine learning algorithms, or hybrid approaches. Finally, **action** refers to the agent's ability to take actions in it

In [37]:

state = graph.get_state(config)

In [38]:

ChatPromptTemplate.from_messages(state.values["messages"]).pretty_print()

================================ Human Message =================================

Generate an article on AI agent design pattern

================================== Ai Message ==================================

**Designing AI Agents: A Comprehensive Guide to the AI Agent Pattern**

Artificial intelligence (AI) has revolutionized the way we interact with machines, and AI agents are at the forefront of this revolution. An AI agent is a software program that perceives its environment, takes actions, and learns from its experiences to achieve a specific goal. In this article, we will delve into the design pattern of AI agents, exploring the key components, benefits, and applications of this powerful paradigm.

At its core, an AI agent consists of three primary components: perception, reasoning, and action. **Perception** involves the agent's ability to gather information from its environment, whether through sensors, APIs, or other data sources. **Reasoning** is the agent's decision-makin